In [1]:
MyRange= spark.range(1000).toDF("number")
divisby2 = MyRange.where("number % 2 = 0 ")
divisby2.count()
#divisby2.show()

Out[ 5 ]: 500

Upload Flight file 2015

In [3]:
flightData2015=spark\
.read\
.option("header","true")\
.option("inferSchema","true")\
.csv("/FileStore/tables/2015_summary-ebaee.csv")

flightData2015.show()

+--------------------+-------------------+-----+
 DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
 United States| Romania| 15|
 United States| Croatia| 1|
 United States| Ireland| 344|
 Egypt| United States| 15|
 United States| India| 62|
 United States| Singapore| 1|
 United States| Grenada| 62|
 Costa Rica| United States| 588|
 Senegal| United States| 40|
 Moldova| United States| 1|
 United States| Sint Maarten| 325|
 United States| Marshall Islands| 39|
 Guyana| United States| 64|
 Malta| United States| 1|
 Anguilla| United States| 41|
 Bolivia| United States| 30|
 United States| Paraguay| 6|
 Algeria| United States| 4|
Turks and Caicos ...| United States| 230|
 United States| Gibraltar| 1|
+--------------------+-------------------+-----+
only showing top 20 rows

In [4]:
flightData2015.sort("count").show()

+--------------------+-------------------+-----+
 DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
 United States| Estonia| 1|
 Kosovo| United States| 1|
 Zambia| United States| 1|
 United States| Papua New Guinea| 1|
 Malta| United States| 1|
 United States| Gibraltar| 1|
 Suriname| United States| 1|
 United States| Croatia| 1|
 Djibouti| United States| 1|
 Burkina Faso| United States| 1|
Saint Vincent and...| United States| 1|
 United States| Cyprus| 1|
 United States| Singapore| 1|
 Moldova| United States| 1|
 Cyprus| United States| 1|
 United States| Lithuania| 1|
 United States| Bulgaria| 1|
 United States| Georgia| 1|
 United States| Bahrain| 1|
 Cote d'Ivoire| United States| 1|
+--------------------+-------------------+-----+
only showing top 20 rows

In [5]:
#oldDF.createOrReplaceTempView("newDF") into table or temp View
flightData2015.createOrReplaceTempView("flight_data_2015") 

In [6]:
sqlWay = spark.sql("""
SELECT DEST_COUNTRY_NAME,count(*)
FROM flight_Data_2015
GROUP BY DEST_COUNTRY_NAME
""")


dataFrameWay = flightData2015\
.groupBy("DEST_COUNTRY_NAME")\
.count()

sqlWay.explain
dataFrameWay.explain

Out[ 31 ]: <bound method DataFrame.explain of DataFrame[DEST_COUNTRY_NAME: string, count: bigint]>

In [7]:
# le max des count
from pyspark.sql.functions import max
flightData2015.select(max("count")).take(1)

Out[ 34 ]: [Row(max(count)=370002)]

In [8]:
# les 5 premiers sum(count) w/ spark.sql
maxSql = spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
FROM flight_Data_2015
GROUP BY DEST_COUNTRY_NAME
ORDER BY sum(count) DESC
LIMIT 5
""")
maxSql.show()

+-----------------+-----------------+
DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
 United States| 411352|
 Canada| 8399|
 Mexico| 7140|
 United Kingdom| 2025|
 Japan| 1548|
+-----------------+-----------------+

In [9]:
# les 5 premiers en utilisant les DFs 
from pyspark.sql.functions import desc
flightData2015\
.groupBy("DEST_COUNTRY_NAME")\
.sum("count")\
.withColumnRenamed("sum(count)","destination_total")\
.sort(desc("destination_total"))\
.limit(5)\
.show()

+-----------------+-----------------+
DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
 United States| 411352|
 Canada| 8399|
 Mexico| 7140|
 United Kingdom| 2025|
 Japan| 1548|
+-----------------+-----------------+